In [21]:
import numpy as np
X = np.random.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB, GaussianNB
clf =  MultinomialNB()
clf.fit(X, y)

test = X[0]
test[1] = 0
test = np.random.randint(5, size=(100))

print(clf.predict_proba(test))

[[  7.94771063e-06   9.90030296e-01   9.86937379e-03   5.95542608e-08
    7.37611133e-05   1.85614682e-05]]


In [47]:
np.ones((10,1))

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [37]:
### normcdf
import numpy as np
from scipy.special import erfc
import math

def normcdfM(x, m=None, s=None):
    if m==None and s==None:
        z = x
    elif s==None:
        z = x-m
    else:
        z = (x-m)/s
        
    return 0.5*erfc(-z/math.sqrt(2))
    

In [78]:
### normpdf

import numpy as np
import math

def normpdfM(x, m=0, s=1):    
    o = 1.0/math.sqrt(2.0*math.pi*(s**2))
    
    return o*math.exp((-((x-m)**2)/(2*(s**2)))[0])
    

In [121]:
##tmean

import numpy as np

def tmean(m, index_max, n_samps):
    
    K = m.shape[0]
    u = np.random.randn(n_samps,1)

    t = m[index_max]*np.ones((K, 1)) - m 
    tr = t
    t = np.delete(t, index_max, 0)
    
    s = np.tile(u, (1, K-1)) + np.tile(t,(1, n_samps)).conj().T
    z = np.mean(np.prod(safenormcdf(s.conj().T), 0))

    tm = np.zeros(m.shape[0])
    for r in range(0, K):
        sr = np.tile(u, (1, K)) + np.tile(tr, (1, n_samps)).conj().T
        sr.take([r, index_max], axis=1) # TEST
        
        #print (u.conj().T + m[index_max]- m[r]).shape
        nr = np.mean(safenormpdf(u.conj().T + m[index_max] 
                                 - m[r])*np.prod(safenormcdf(sr.conj().T), 1))
        
        if r == index_max:
            tm[r] = 0
        else:
            tm[r] = m[r] - nr/z
            
    tm[index_max] = np.sum(m, axis=0) - np.sum(tm, axis=0)
    tm = tm.conj().T                    
            
    return tm, z
                                                               

def safenormcdf(x):
    #x = x[0]
    thresh=-10;
    x[np.nonzero(x<thresh)] = thresh
    return normcdfM(x)

def safenormpdf(x):
    x=x[0]
    thresh=35;
    x[np.nonzero(x<-thresh)] = -thresh
    x[np.nonzero(x>thresh)] = thresh
    return normpdfM(x)

def safelog(x):
    x[np.nonzero(x<1e-300)] = 1e-200
    x[np.nonzero(x>1e300)] = 1e300
    return math.log(x)
        

In [172]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

def dist(X,Y,Z): # X, Y matrices, Z scalar
    
    nx = X.shape[0]
    ny = Y.shape[0]
    
    vector = np.sum((X**2)*Z, 1).conj().T
    vector = vector.reshape(1, vector.shape[0])

    distance= (np.sum((X**2)*Z, 1)*np.ones((1,ny))+
              np.ones((nx,1))*(np.sum((X**2)*Z, 1).conj().T) - 2*(np.dot(X*Z, Y.conj().T))) ## NOT WORKING

    #print np.shape(np.ones((nx,1))*vector)
    #distance= (np.sum((X**2)*Z, 1)*np.ones((1,ny))+
    #       np.ones((nx,1))*vector - 2*(np.dot(X*Z, Y.conj().T)))
           
    return distance


def VarMultProbRegGP(X, t, X_test=0, t_test=0, theta=0, theta_estimate=0, 
                     nos_its=0, kernel_Type=0, poly_kernel_power=0, thresh=0):

    SMALL_NOS = 1e-10

    nos_samps_tg = 1000

    nos_samps_is = 1000

    sigma = 1e-6

    tau = 1e-6

    C = np.max(t) # change to number of classes
    N, D = X.shape
    Y = np.random.randn(N, C)
    M = np.random.rand(N, C)
    beta = theta
    Theta = np.diag(theta)

    psi = np.ones((1, theta.size)) 
    In = np.eye(N)
    Ic = np.eye(C)

    diff = 1e100
    its = 0

    #K = np.exp(-dist(X,X,Theta)) + np.eye(N)*SMALL_NOS; # TODO: implement create kernel function
    K = rbf_kernel(X,X, Theta)+ np.eye(N)*SMALL_NOS
    
    iK = np.linalg.inv(K+In)
    Ki = K*iK
    
    THETA = []
    #LOWER_BOUND = [-1e-3]
    PL = []
    
    test_err = []
    while its<nos_its and diff>thresh:
        its += 1
        
        for k in range(0, C):
            M[:,k] = np.dot(Ki, Y[:,k])
        
        #print M.shape
        for n in range(0, N):
            #print t[n]
            a, z = tmean(M[[n],].conj().T, t[n]-1, nos_samps_tg)
            Y[n,] = a.conj().T
            
        #print Y.shape
        
    
        ####### predict ########

        n_test = X_test.shape[0]
        #K_test = np.exp(-dist(X,X_test,Theta))
        #K_test_self = np.exp(-dist(X_test,X_test,Theta))
        K_test = rbf_kernel(X,X_test,Theta)
        K_test_self = rbf_kernel(X_test,X_test,Theta)

        S = (np.diag(K_test_self) - np.diag(np.dot(np.dot(K_test.conj().T, iK), K_test))).conj().T

        predictive_likelihood = 0
        res = (np.dot(np.dot(Y.conj().T, iK), K_test)).conj().T
        P_test = np.ones((n_test, C))
        u = np.random.randn(nos_samps_tg, 1)

        for n in range(0, n_test):
            for i in range(0, C):
                pp = np.ones((nos_samps_tg, 1))
                for j in range(0, C):
                    if j!=i:
                        pp = pp*safenormcdf(u+(res[n,i]-res[n,j])/(math.sqrt(1+S[n])))               

                P_test[n,i] = np.mean(pp)
            P_test[n,] = P_test[n,]/np.sum(P_test[n,])

            #predictive_likelihood = predictive_likelihood + safelog(P_test[n, t_test[n]])

        P_test = P_test.conj().T

    
    print P_test
    print res
        
    
X = np.random.randint(5, size=(6, 100))
X_test = np.random.randint(5, size=(2, 100))
theta = np.random.rand(1, 10)
y = np.array([0, 1, 2, 3, 4, 5])
VarMultProbRegGP(X, y, theta=theta, nos_its=100, thresh = 0.01, X_test=X_test)    



[[ 0.2  0.2]
 [ 0.2  0.2]
 [ 0.2  0.2]
 [ 0.2  0.2]
 [ 0.2  0.2]]
[[  1.08007194e-105  -3.40203937e-107   1.81982278e-106   1.10179554e-106
   -3.60483426e-106]
 [ -1.37709308e-119  -6.36713805e-119   3.54825053e-119   3.31186403e-118
   -1.30828747e-118]]
